In [ ]:
import os
import torch
import sys
from PIL import Image
import numpy as np
import pandas as pd
import cv2
from google.colab import drive
from google.colab.patches import cv2_imshow
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/Nam4_HK1/NLN/Web

!pip install -r requirements.txt 

!pip install --quiet vietocr==0.3.5

!pip install flask-ngrok
!pip install flask==0.12.2
!pip install pyngrok

# restart runtime
os.kill(os.getpid(), 9)

In [ ]:
# Import
py_file_location = "/content/drive/My Drive/Nam4_HK1/NLN/Web/app/templates/utils"
sys.path.append(os.path.abspath(py_file_location))
%cd /content/drive/My Drive/Nam4_HK1/NLN/Web/app/templates/utils
import crop_image
import detect_word

/content/drive/My Drive/Nam4_HK1/NLN/Web/app/templates/utils


In [ ]:
%cd /content/drive/My Drive/Nam4_HK1/NLN/Web/app/templates/model
#Load model
#Load model cropper
model_crop = torch.hub.load('ultralytics/yolov5', 'custom', path='model_crop.pt')

#Load model detecter
model_detect = torch.hub.load('ultralytics/yolov5', 'custom', path='model_detect.pt')

#Load model OCR (reader)
from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg
config = Cfg.load_config_from_name('vgg_transformer')
config['weights'] = 'transformerocr.pth'
config['cnn']['pretrained']=False
config['device'] = 'cpu'
config['predictor']['beamsearch']=False

detector = Predictor(config)

/content/drive/My Drive/Nam4_HK1/NLN/Web/app/templates/model


Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip


YOLOv5 🚀 2021-12-16 torch 1.10.0+cu111 CPU

Fusing layers... 
Model Summary: 476 layers, 87218881 parameters, 0 gradients
Adding AutoShape... 
Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-12-16 torch 1.10.0+cu111 CPU

Fusing layers... 
Model Summary: 367 layers, 46129818 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
%cd /content/drive/My Drive/Nam4_HK1/NLN/Web/app

from flask import Flask, flash, request, redirect, url_for, render_template
import urllib.request
import os
import threading
from   import ngrok

from werkzeug.utils import secure_filename

# # run_with_ngrok
# from flask_ngrok import run_with_ngrok 
# app = Flask(__name__)
# run_with_ngrok(app)

os.environ["FLASK_ENV"] = "development"
app = Flask(__name__)
port = 5000
# auth_token  -  https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token("21zbTIa8yGKu6OJKPUHuyPAjmjV_ENV89BHoMoSbKjFA9ZBz")
# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url

print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))
# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url  


UPLOAD_FOLDER = 'static/uploads/'
 
app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024
 
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg', 'gif'])
 
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
     
 
@app.route('/')
def home():
    return render_template('index.html')

#Get image and OCR
@app.route('/', methods=['POST'])
def upload_image():
    if 'file' not in request.files:
        flash('No file part')
        return redirect(request.url)
    file = request.files['file']
    if file.filename == '':
        flash('No image selected for uploading')
        return redirect(request.url)
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        #print('upload_image filename: ' + filename)

#OCR
        image = cv2.imread(os.path.join('static/uploads',filename))
        # cv2_imshow(image)
        # predict (Cropper)
        results_crop = model_crop(image, size=640)
        # crop
        try:
          for j in range(len(results_crop.xyxy)):
            img_crop = crop_image.CropImg(results_crop.xyxy[j],results_crop.pandas().xyxy[j],image)
        except BaseException:
          return render_template('index.html', filename=filename, id ="None", name ="None", birth ="None", home ="None", add ="None")
        # Detector
        try:
          results_detect = model_detect(img_crop, size=640)
        except BaseException:
          return render_template('index.html', filename=filename, id ="None", name ="None", birth ="None", home ="None", add ="None")
        #OCR
        try:
          for j in range(len(results_detect.xyxy)):
            dic = detect_word.OCR(results_detect.xyxy[j],results_detect.pandas().xyxy[j],img_crop, detector)
        except BaseException:
          return render_template('index.html', filename=filename, id ="None", name ="None", birth ="None", home ="None", add ="None")

        # print(dic)
        #miss infor
        if "id" not in dic:
          dic['id'] = 'None'
        if "name" not in dic:
          dic['name'] = 'None'
        if "birth" not in dic:
          dic['birth'] = 'None'
        if "home" not in dic:
          dic['home'] = 'None'
        if "add" not in dic:
          dic['add'] = 'None'
 
        print(dic)
        return render_template('index.html', filename=filename, id =dic["id"], name =dic["name"], birth =dic["birth"], home =dic["home"], add =dic["add"])
    
    else:
        flash('Allowed image types are - png, jpg, jpeg, gif')
        return redirect(request.url)
 

# Load image
@app.route('/display/<filename>')
def display_image(filename):
    #print('display_image filename: ' + filename)
    return redirect(url_for('static', filename='uploads/' + filename), code=301)

# # run_with_ngrok 
# if __name__ == "__main__":
#     app.run()

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

/content/drive/My Drive/Nam4_HK1/NLN/Web/app


Updating authtoken for default "config_path" of "ngrok_path": /usr/local/lib/python3.7/dist-packages/pyngrok/bin/ngrok
Opening tunnel named: http-5000-74e57542-a778-4f85-b349-d76c2dbfe1f5
t=2021-12-16T05:43:03+0000 lvl=info msg="no configuration paths supplied"
t=2021-12-16T05:43:03+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
t=2021-12-16T05:43:03+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
t=2021-12-16T05:43:03+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040
t=2021-12-16T05:43:04+0000 lvl=info msg="tunnel session started" obj=tunnels.session
t=2021-12-16T05:43:04+0000 lvl=info msg="client session established" obj=csess id=03436198c121
t=2021-12-16T05:43:04+0000 lvl=info msg=start pg=/api/tunnels id=0884b0875eba1370
t=2021-12-16T05:43:04+0000 lvl=info msg=end pg=/api/tunnels id=0884b0875eba1370 status=200 dur=572.448µs
t=2021-12-16T05:43:04+0000 lvl=info msg=start pg=/api/tunnels id=acf3d

 * ngrok tunnel "http://e291-34-73-225-179.ngrok.io" -> "http://127.0.0.1:5000"


t=2021-12-16T05:43:04+0000 lvl=info msg=start pg="/api/tunnels/http-5000-74e57542-a778-4f85-b349-d76c2dbfe1f5 (http)" id=52ea839d4de699ba
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
t=2021-12-16T05:43:04+0000 lvl=info msg=end pg="/api/tunnels/http-5000-74e57542-a778-4f85-b349-d76c2dbfe1f5 (http)" id=52ea839d4de699ba status=200 dur=198.144µs
